In [25]:
# Load required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [27]:
import os

try:
    from google.colab import drive

    # Mount Google Drive
    drive.mount('/content/drive')

    # Set working directory
    os.chdir('/content/drive/My Drive/CSC 461/Final/')

    # Extract images from train.tar. Takes about 11 minutes.
    # Has to be done each time you restart the runtime unless you extract and upload
    # the files manually (which can take hours)
    !tar -xf train.tar --totals --checkpoint=.4000 --checkpoint-action='echo=Hit %s checkpoint #%u/79433'
except:
    # If it fails, we're probably running locally and don't need to extract the files
    pass

In [ ]:
# Load the data
df = pd.read_csv('train_info.csv')

# Keep only the columns we need: filename and style
df = df[['filename', 'style']]

# Drop rows with blank styles
df = df.dropna()

# Drop rows with styles that have less than 500 images
df = df[df['style'].map(df['style'].value_counts()) > 500]

# Show counts for each style
# df['style'].value_counts()

In [ ]:
# Split the data into train and test sets with a 90/10 split and stratify by style
train, test = train_test_split(df, test_size=0.1, random_state=42, stratify=df['style'])

# Split the train set into train and validation sets with an 89/11 split and stratify by style
# This will result in an overall split of 80.1/9.9/10 for train/validation/test
train, val = train_test_split(train, test_size=0.11, random_state=42, stratify=train['style'])